In [2]:
import pandas as pd
import os
import shutil
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# CSV 경로
TRAIN_CSV = r'D:\학교\3-2\파이썬기반 딥러닝\3-2_jester_recognation\jester_dataset_v1\Train.csv'
VAL_CSV   = r'D:\학교\3-2\파이썬기반 딥러닝\3-2_jester_recognation\jester_dataset_v1\Validation.csv'

# 원본 데이터셋 최상위 경로 
SOURCE_DIR = r'D:\학교\3-2\파이썬기반 딥러닝\3-2_jester_recognation\jester_dataset_v1'
SOURCE_SUBFOLDERS = ['Train', 'Validation', 'Test']

# 정리된 데이터셋을 저장할 경로
TARGET_DIR = r'D:\학교\3-2\파이썬기반 딥러닝\3-2_jester_recognation\jester_final_dataset'

TARGET_LABELS = [
    # 1. 기본 배경 (오작동 방지용 필수)
    "No gesture",
    "Doing other things",

    # 2. 정밀 줌 제어 (지도 확대/축소 등 디테일한 조작)
    "Zooming In With Two Fingers",
    "Zooming Out With Two Fingers",

    # 3. 전체 줌/강조 (이미지 전체 크기 조절 등 큰 조작)
    "Zooming In With Full Hand",
    "Zooming Out With Full Hand",

    # 4. 의사 표현 / 확인 / 취소 (UI 버튼 클릭 대신 사용)
    "Thumb Up",    # 승인, 좋아요, Yes
    "Thumb Down"   # 거절, 싫어요, No
]

COPY_MODE = True
MAX_WORKERS = 24  # 동시에 작업할 스레드 개수 (CPU 코어 수 * 2~4 권장, 너무 높으면 디스크 병목 발생)

def find_video_source_path(video_id):
    """SOURCE_DIR 하위 폴더들을 뒤져서 영상 경로를 찾음"""
    candidates = [video_id, video_id.zfill(5)] 
    for subfolder in SOURCE_SUBFOLDERS:
        for vid_candidate in candidates:
            candidate_path = os.path.join(SOURCE_DIR, subfolder, vid_candidate)
            if os.path.exists(candidate_path):
                return candidate_path
    return None

def copy_worker(row_data):
    """
    개별 영상 하나를 처리하는 작업자 함수 (병렬 실행됨), 반환값: (성공여부(True/False), 메세지/사유)
    """
    video_id, label, split_name = row_data
    
    src_path = find_video_source_path(video_id)  # 원본 찾기
    if not src_path:
        return (False, "missing") # 원본 없음

    dest_path = os.path.join(TARGET_DIR, split_name, label, video_id) # 타겟 경로 설정
    if os.path.exists(dest_path):
        return (True, "exists") # 이미 존재함

    try:
        os.makedirs(dest_path, exist_ok=True)
        if COPY_MODE:
            shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
        else:
            shutil.move(src_path, dest_path)
        return (True, "copied")
    except Exception as e:
        return (False, str(e)) # 에러 발생

def process_split(csv_path, split_name):
    if not os.path.exists(csv_path):
        print(f"{csv_path} 파일이 없습니다. 건너뜁니다.")
        return

    print(f"\n{split_name} 데이터셋 처리 중 (병렬 처리)...")
    
    try:
        df = pd.read_csv(csv_path, sep=None, engine='python', header=0, dtype={'video_id': str, 'label': str})
        df.columns = df.columns.str.strip()
        '''if 'video_id' not in df.columns or 'label' not in df.columns:
            print(f"필수 컬럼 누락. 감지된 컬럼: {list(df.columns)}")
            return'''
    except Exception as e:
        print(f"CSV 읽기 실패: {e}")
        return
    
    # 작업 목록 생성 (데이터 전처리)
    tasks = []
    skipped_label_count = 0
    
    print("작업 목록 생성 중...")
    for _, row in df.iterrows():
        video_id = str(row['video_id']).strip()
        label = str(row['label']).strip()
        
        if label in TARGET_LABELS:
            # 병렬 처리를 위해 필요한 정보를 튜플로 묶어서 리스트에 담음
            tasks.append((video_id, label, split_name))
        else:
            skipped_label_count += 1

    print(f"-> 총 {len(df)}개 중 타겟 데이터 {len(tasks)}개 (제외됨: {skipped_label_count}개)")
    
    # 병렬 처리 시작
    success_count = 0
    missing_count = 0
    error_count = 0
    
    # ThreadPoolExecutor로 병렬 실행
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # 작업을 스레드에 등록하고 결과(Future)를 받음
        futures = [executor.submit(copy_worker, task) for task in tasks]
        
        # tqdm으로 진행률 표시 (as_completed는 작업이 끝나는 순서대로 반환)
        for future in tqdm(as_completed(futures), total=len(tasks), desc=f"Copying {split_name}"):
            is_success, msg = future.result()
            
            if is_success:
                success_count += 1
            else:
                if msg == "missing":
                    missing_count += 1
                else:
                    error_count += 1
                    # 에러가 너무 많이 뜨면 로그가 지저분해지므로 주석 처리하거나 필요시 해제
                    # print(f"Error: {msg}")

    print(f"{split_name} 결과:")
    print(f"- 성공(저장/이미존재): {success_count}개")
    print(f"- 실패(원본 못찾음): {missing_count}개")
    print(f"- 에러: {error_count}개")

def main():
    print(f"타겟 라벨 {len(TARGET_LABELS)}개에 대해 병렬 작업을 시작합니다.")
    print(f"사용 스레드 수: {MAX_WORKERS}")
    
    if not os.path.exists(TARGET_DIR):
        os.makedirs(TARGET_DIR)

    process_split(TRAIN_CSV, 'train')
    process_split(VAL_CSV, 'val')

    print("\n" + "="*40)
    print(f"모든 작업 완료! 저장 위치: {os.path.abspath(TARGET_DIR)}")
    print("="*40)

main()

타겟 라벨 8개에 대해 병렬 작업을 시작합니다.
사용 스레드 수: 24

train 데이터셋 처리 중 (병렬 처리)...
작업 목록 생성 중...
-> 총 50420개 중 타겟 데이터 17148개 (제외됨: 33272개)


Copying train: 100%|██████████| 17148/17148 [09:05<00:00, 31.43it/s]


train 결과:
- 성공(저장/이미존재): 17148개
- 실패(원본 못찾음): 0개
- 에러: 0개

val 데이터셋 처리 중 (병렬 처리)...
작업 목록 생성 중...
-> 총 7047개 중 타겟 데이터 2497개 (제외됨: 4550개)


Copying val: 100%|██████████| 2497/2497 [01:21<00:00, 30.47it/s]

val 결과:
- 성공(저장/이미존재): 2497개
- 실패(원본 못찾음): 0개
- 에러: 0개

모든 작업 완료! 저장 위치: D:\학교\3-2\파이썬기반 딥러닝\3-2_jester_recognation\jester_final_dataset
